In [1]:
# ! pip install kaggle
# ! kaggle datasets download -d eliasdabbas/web-server-access-logs
# ! unzip web-server-access-logs.zip
# ! rm web-server-access-logs.zip

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("spark://spark-master:7077") \
.config("spark.executor.memory", "2g") \
.config("spark.executor.instances", "6") \
.config("spark.executor.cores", "2") \
.config("spark.driver.memory", "4g") \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/27 07:56:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.setLogLevel("ERROR")

In [4]:
from pyspark.sql.functions import regexp_extract, col, to_timestamp
from pyspark.sql.types import LongType

import re
import os
import time
from tqdm import tqdm

In [5]:
log_file_path = "/opt/workspace/access_logs/access.log"

combined_regex = r'^(\S+) \S+ (\S+) \[([^\]]+)\] "([A-Z]+) ([^ "]+)? HTTP/[0-9.]+" ([0-9]{3}) ([0-9]+|-) "([^"]*)" "([^"]*)'
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']

In [6]:
web_logs = spark.read.text(log_file_path)

In [7]:
web_logs_raw = web_logs.select(
    regexp_extract("value", combined_regex, 1).alias(columns[0]),
    regexp_extract("value", combined_regex, 2).alias(columns[1]),
    regexp_extract("value", combined_regex, 3).alias(columns[2]),
    regexp_extract("value", combined_regex, 4).alias(columns[3]),
    regexp_extract("value", combined_regex, 5).alias(columns[4]),
    regexp_extract("value", combined_regex, 6).alias(columns[5]),
    regexp_extract("value", combined_regex, 7).alias(columns[6]),
    regexp_extract("value", combined_regex, 8).alias(columns[7]),
    regexp_extract("value", combined_regex, 9).alias(columns[8]),
)

In [8]:
web_logs_raw.show()

+-------------+------+--------------------+------+--------------------+------+-----+--------------------+--------------------+
|       client|userid|            datetime|method|             request|status| size|             referer|          user_agent|
+-------------+------+--------------------+------+--------------------+------+-----+--------------------+--------------------+
| 54.36.149.41|     -|22/Jan/2019:03:56...|   GET|/filter/27|13%20%...|   200|30577|                   -|Mozilla/5.0 (comp...|
|  31.56.96.51|     -|22/Jan/2019:03:56...|   GET|/image/60844/prod...|   200| 5667|https://www.zanbi...|Mozilla/5.0 (Linu...|
|  31.56.96.51|     -|22/Jan/2019:03:56...|   GET|/image/61474/prod...|   200| 5379|https://www.zanbi...|Mozilla/5.0 (Linu...|
|40.77.167.129|     -|22/Jan/2019:03:56...|   GET|/image/14925/prod...|   200| 1696|                   -|Mozilla/5.0 (comp...|
|  91.99.72.15|     -|22/Jan/2019:03:56...|   GET|/product/31893/62...|   200|41483|                   -|Mozill

In [9]:
web_logs_raw.count()

10365152

In [10]:
condition = col(columns[0]) == ''
for column in columns[1:]:
    condition &= col(column) == ''

In [11]:
web_logs_raw.filter(condition).count()

287

In [12]:
web_logs_df =  web_logs_raw.filter(~condition)

In [13]:
web_logs_df.count()

10364865

In [14]:
web_logs_df.select("datetime").filter("client = '54.36.149.41'").limit(1).show(truncate=False)

+--------------------------+
|datetime                  |
+--------------------------+
|22/Jan/2019:03:56:14 +0330|
+--------------------------+



In [15]:
web_logs_df.printSchema()

root
 |-- client: string (nullable = true)
 |-- userid: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- method: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status: string (nullable = true)
 |-- size: string (nullable = true)
 |-- referer: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [16]:
web_logs_df = web_logs_df.withColumn("status", col("status").cast(LongType())) \
            .withColumn("size", col("size").cast(LongType()))

In [17]:
web_logs_df.printSchema()

root
 |-- client: string (nullable = true)
 |-- userid: string (nullable = true)
 |-- datetime: string (nullable = true)
 |-- method: string (nullable = true)
 |-- request: string (nullable = true)
 |-- status: long (nullable = true)
 |-- size: long (nullable = true)
 |-- referer: string (nullable = true)
 |-- user_agent: string (nullable = true)



In [18]:
web_logs_df.show()

+-------------+------+--------------------+------+--------------------+------+-----+--------------------+--------------------+
|       client|userid|            datetime|method|             request|status| size|             referer|          user_agent|
+-------------+------+--------------------+------+--------------------+------+-----+--------------------+--------------------+
| 54.36.149.41|     -|22/Jan/2019:03:56...|   GET|/filter/27|13%20%...|   200|30577|                   -|Mozilla/5.0 (comp...|
|  31.56.96.51|     -|22/Jan/2019:03:56...|   GET|/image/60844/prod...|   200| 5667|https://www.zanbi...|Mozilla/5.0 (Linu...|
|  31.56.96.51|     -|22/Jan/2019:03:56...|   GET|/image/61474/prod...|   200| 5379|https://www.zanbi...|Mozilla/5.0 (Linu...|
|40.77.167.129|     -|22/Jan/2019:03:56...|   GET|/image/14925/prod...|   200| 1696|                   -|Mozilla/5.0 (comp...|
|  91.99.72.15|     -|22/Jan/2019:03:56...|   GET|/product/31893/62...|   200|41483|                   -|Mozill

In [23]:
web_logs_df.write.parquet("/opt/workspace/access_logs/access_logs.parquet")

24/07/27 07:59:56 ERROR TaskSchedulerImpl: Lost executor 7 on 172.19.0.6: Command exited with code 137
24/07/27 08:00:15 ERROR TaskSchedulerImpl: Lost executor 8 on 172.19.0.4: Command exited with code 137
24/07/27 08:00:31 ERROR TaskSchedulerImpl: Lost executor 9 on 172.19.0.7: Command exited with code 137
24/07/27 08:00:45 ERROR TaskSchedulerImpl: Lost executor 10 on 172.19.0.6: Command exited with code 137
24/07/27 08:00:58 ERROR TaskSchedulerImpl: Lost executor 11 on 172.19.0.4: Command exited with code 137


In [24]:
t = spark.read.parquet("/opt/workspace/access_logs/access_logs.parquet")

In [25]:
t.show()

+-------------+------+--------------------+------+--------------------+------+-----+--------------------+--------------------+
|       client|userid|            datetime|method|             request|status| size|             referer|          user_agent|
+-------------+------+--------------------+------+--------------------+------+-----+--------------------+--------------------+
| 37.148.52.71|     -|25/Jan/2019:00:10...|   GET|/static/bundle-bu...|   200|55531|https://www.zanbi...|Mozilla/5.0 (Wind...|
|   5.232.1.88|     -|25/Jan/2019:00:10...|   GET|/image/21555?name...|   200| 5452|https://www.googl...|Mozilla/5.0 (Linu...|
|207.46.13.137|     -|25/Jan/2019:00:10...|   GET|/filter/b171,b36,...|   200|37659|                   -|Mozilla/5.0 (comp...|
|46.224.205.67|     -|25/Jan/2019:00:10...|   GET|      /settings/logo|   200| 4120|https://www.zanbi...|Mozilla/5.0 (iPho...|
| 37.148.52.71|     -|25/Jan/2019:00:10...|   GET|/static/images/se...|   200|  217|https://znbl.ir/s...|Mozill

In [27]:
! du -h /opt/workspace/access_logs/access_logs.parquet/

273M	/opt/workspace/access_logs/access_logs.parquet/
